<a href="https://colab.research.google.com/github/sushil79g/health_convbot/blob/master/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
base_folder = 'drive/My Drive/dataset/'
!pip install contractions
!pip install textsearch

     |████████████████████████████████| 317kB 8.9MB/s 
     |████████████████████████████████| 245kB 39.5MB/s 
  Stored in directory: /root/.cache/pip/wheels/0a/90/61/87a55f5b459792fbb2b7ba6b31721b06ff5cf6bde541b40994
Successfully built pyahocorasick


In [3]:
# -*- coding: utf-8 -*-
"""
Spyder Editor

This is a temporary script file.
"""
import re
import pandas as pd
import contractions

contractions.add("c'mon", 'come on')

file = pd.read_csv(base_folder + 'friends_chat.txt',sep='\t')
lines = file['line']

print('Initial chats')
for i in range(10):
    print(lines[i])
    
def cleanchat(line):
    #to convert text to lower case
    line = line.lower()
    #to remove ending EOL
    line = re.sub(r'\n','',line) 
    #re-format punctuations
    line = re.sub(r"[-()]", "", line)
    line = re.sub(r"\.", " .", line)
    line = re.sub(r"\!", " !", line)
    line = re.sub(r"\?", " ?", line)
    line = re.sub(r"\,", " ,", line)
    
    #string replacement
    line = re.sub(r"i'm", "i am", line)
    line = re.sub(r"he's", "he is", line)
    line = re.sub(r"she's", "she is", line)
    line = re.sub(r"it's", "it is", line)
    line = re.sub(r"that's", "that is", line)
    line = re.sub(r"what's", "that is", line)
    line = re.sub(r"\'ll", " will", line)
    line = re.sub(r"\'re", " are", line)
    line = re.sub(r"won't", "will not", line)
    line = re.sub(r"can't", "cannot", line)
    line = re.sub(r"n't", " not", line)
    line = re.sub(r"n'", "ng", line)
    line = re.sub(r"ohh", "oh", line)
    line = re.sub(r"ohhh", "oh", line)
    line = re.sub(r"ohhhh", "oh", line)
    line = re.sub(r"ohhhhh", "oh", line)
    line = re.sub(r"ohhhhhh", "oh", line)
    line = re.sub(r"ahh", "ah", line)
    
    return line
    

Initial chats
There's nothing to tell! He's just some guy I work with!
C'mon, you're going out with the guy! There's gotta be something wrong with him!
Alright Joey, be nice. So does he have a hump? A hump and a hairpiece?
Wait, does he eat chalk?
Just, 'cause, I don't want her to go through what I went through with Carl- oh!
Okay, everybody relax. This is not even a date. It's just two people going out to dinner and not having sex.
Sounds like a date to me.
Alright, so I'm back in high school, I'm standing in the middle of the cafeteria, and I realize I am totally naked.
Oh, yeah. Had that dream.
Then I look down, and I realize there's a phone... there.


In [0]:
def preprocess(x):
    clean = cleanchat(x)
    processed = contractions.fix(clean)
    return processed

file['processed_text'] = file['line'].apply(preprocess)


In [0]:
def length(x):
    return len(x.split())
file['length'] = file['line'].apply(length)

In [7]:
file['length'].describe()

count    60849.000000
mean        10.161416
std         10.438841
min          1.000000
25%          3.000000
50%          7.000000
75%         14.000000
max        195.000000
Name: length, dtype: float64

In [0]:
MAX_WORD_LENGTH = 15

In [9]:
file.head(5)

,id,scene_id,person,gender,original_line,line,metadata,filename,processed_text,length
0,1,1,MONICA,F,Monica: There's nothing to tell! He's just som...,There's nothing to tell! He's just some guy I ...,There_EX 's_VBZ nothing_PN1 to_TO tell_VVI !_!...,0101.txt,there is nothing to tell ! he is just some guy...,11
1,101,1,JOEY,M,"Joey: C'mon, you're going out with the guy! Th...","C'mon, you're going out with the guy! There's ...",C'm_VV0 on_RP you_PPY 're_VBR going_VVG out_RP...,0101.txt,"come on , you are going out with the guy ! the...",14
2,201,1,CHANDLER,M,"Chandler: All right Joey, be nice. So does he...","Alright Joey, be nice. So does he have a hump?...",All_RR21 right_RR22 Joey_NP1 be_VBI nice_JJ ._...,0101.txt,"alright joey , be nice . so does he have a hum...",15
3,301,1,PHOEBE,F,"Phoebe: Wait, does he eat chalk?","Wait, does he eat chalk?",Wait_VV0 does_VDZ he_PPHS1 eat_VVI chalk_NN1 ?_?,0101.txt,"wait , does he eat chalk ?",5
4,401,1,PHOEBE,F,"Phoebe: Just, 'cause, I don't want her to go t...","Just, 'cause, I don't want her to go through w...",Just_RR 'cause_CS I_PPIS1 do_VD0 n't_XX want_V...,0101.txt,"just , because , i do not want her to go throu...",16


In [10]:
file.isnull().sum()

id                0
scene_id          0
person            0
gender            0
original_line     0
line              0
metadata          0
filename          0
processed_text    0
length            0
dtype: int64

In [0]:
temp_dataframe = file[['person','processed_text']]
remove_index = []
chat_pair = []
i = 0
while i <= (len(temp_dataframe)-2):
    raw_i = i
    
    while True:
#         print(raw_i,i, i+1)
        if temp_dataframe.iloc[i+1]['person'] == temp_dataframe.iloc[i]['person']:
            temp_dataframe.iloc[raw_i]['processed_text'] += ','+ temp_dataframe.iloc[i+1]['processed_text']
            chat_pair.append([temp_dataframe.iloc[raw_i]['processed_text'], temp_dataframe.iloc[raw_i+1]['processed_text']])
            remove_index.append(i+1)
        else:
            break
        i = i+1
        
    i += 1

In [12]:
temp_dataframe.drop(remove_index, inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [13]:
temp_dataframe.head(5)

,person,processed_text
0,MONICA,there is nothing to tell ! he is just some guy...
1,JOEY,"come on , you are going out with the guy ! the..."
2,CHANDLER,"alright joey , be nice . so does he have a hum..."
3,PHOEBE,"wait , does he eat chalk ?,just , because , i ..."
5,MONICA,"okay , everybody relax . this is not even a da..."


In [0]:
# contractions.add("c'mon", 'come on')
list_of_word = list(temp_dataframe['processed_text'])

In [0]:
words = ' '.join(list_of_word).split()

In [16]:
len(words)

836085

In [0]:
from collections import Counter
wordcount = Counter(words)

In [0]:
vocab= set(words)

In [19]:
len(vocab)

17828

In [0]:
word_threshold = []
for word in vocab:
    if wordcount[word] >=3:
        word_threshold.append(word)
#     break

In [0]:
# word2ind ={{word, index} for word,index in enumerate(list(word_threshold))}

In [0]:
word2index = {}
index2word = {}
for index,word in enumerate(word_threshold):
    word2index.update({word:index})
    index2word.update({index:word})

In [22]:
from pprint import pprint
# pprint('total word in vocab',len(word_threshold))
len(word_threshold)

6756

In [46]:
len(set(word2index.keys()))

6756

In [24]:
chat_pair[1]

['sounds like a date to me .,alright , so i am back in high school , i am standing in the middle of the cafeteria , and i realize i am totally naked .',
 'alright , so i am back in high school , i am standing in the middle of the cafeteria , and i realize i am totally naked .']

In [0]:
def find_max_length(sentences):
    whole_sentence = []
    for sentence in sentences:
        sentence_remove = []
        for word in sentence.split():
            if word in word2index.keys():
                sentence_remove.append(word)
        whole_sentence.append(' '.join(sentence_remove))
    return whole_sentence, max([len(sent) for sent in whole_sentence])



In [0]:
whole_sentence, max_length = find_max_length(list_of_word)

In [29]:
length = []
for sent in whole_sentence:
    try:
        len_ = len(sent.split())
        length.append(len_)
        if len_ == 259:
            print(sent)    
    except:
        print(sent)
max_length = max(length)
max_length

okay it is time for the toast ! umm now now , i know that ross usually gives the toast , but this year i am going to do it .,no , no it is going to be great . really ! mom , dad , when i got married , one of the things that made me sure i could do it was the amazing example the two of you set for me . for that and so many other things i want to say thank you . i know i probably do not say it enough , but i love you . when i look around this room , i am by the thought of those who could not be here with us . nana , my beloved grandmother who would so want to be here , but she cannot because she is dead . as is our dog chichi . i mean look how cute she is . . was . do me a favor and pass this to my parents . remember she is dead . okay , her and nana , gone . wow ! hey does anybody remember when had to say goodbye to her children in terms of ? did not see that ? no movie fans ? ! you want to hear something sad ? the other day i was watching 60 minutes these in , who have been so , they w

259

In [30]:
# EOS_token = -1
# def make_equal_length(total_tensor):
#     max_length = max([len(sent) for sent in total_tensor])
#     for sent in total_tensor:
#         if len(sent) < max_length:
            
    
# def input_process(input_data, word2index):
#     total_tensor = []
#     max_input_length = max([len(sent.split()) for sent in input_data])
#     for input_sent in input_data:
#         tensor = [0] * max_input_length
#         tensor = [tensor.insert() for index,word in enumerate(input_sent)] + [EOS_token]
#         total_tensor.append(tensor)
#     format_input = make_equal_length(total_tensor)
    

# def prepare_batch(batches_group, word2index, index2word):
#     input_data = [batch[0] for batch in batches_group]
#     output_data = [batch[1] for batch in bathces]
#     length = max( len(sent.split()) for sent in input_data)
#     input_processed = input_process(input_data, word2index)


# small_batch = 5
# batches_group = [random.choice(chat_pair) for _num in range(small_batch)]

IndentationError: ignored

In [31]:
import random
import torch
import itertools

EOS_token = -1
PAD_token = 0
def indexesFromSentence(word2index, sentence):
#     print(sentence)
    listdata = []
    for word in sentence.split():
        try:
            listdata.append(word2index[word])
        except:
            pass
    return_list = listdata + [EOS_token]
    return return_list


def zeroPadding(l, fillvalue=PAD_token):
    return list(itertools.zip_longest(*l, fillvalue=fillvalue))

def binaryMatrix(l, value=PAD_token):
    m = []
    for i, seq in enumerate(l):
        m.append([])
        for token in seq:
            if token == PAD_token:
                m[i].append(0)
            else:
                m[i].append(1)
    return m

# Returns padded input sequence tensor and lengths
def inputVar(l, word2index):
    indexes_batch = [indexesFromSentence(word2index, sentence) for sentence in l]
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    padVar = torch.LongTensor(padList)
    return padVar, lengths

# Returns padded target sequence tensor, padding mask, and max target length
def outputVar(l, word2index):
    indexes_batch = [indexesFromSentence(word2index, sentence) for sentence in l]
    max_target_len = max([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    mask = binaryMatrix(padList)
    mask = torch.ByteTensor(mask)
    padVar = torch.LongTensor(padList)
    return padVar, mask, max_target_len

# Returns all items for a given batch of pairs
def batch2TrainData(word2index, pair_batch):
    pair_batch.sort(key=lambda x: len(x[0].split(" ")), reverse=True)
    input_batch, output_batch = [], []
    for pair in pair_batch:
        input_batch.append(pair[0])
        output_batch.append(pair[1])
    inp, lengths = inputVar(input_batch, word2index)
    output, mask, max_target_len = outputVar(output_batch, word2index)
    return inp, lengths, output, mask, max_target_len


# Example for validation
small_batch_size = 5
batches = batch2TrainData(word2index, [random.choice(chat_pair) for _ in range(small_batch_size)])
input_variable, lengths, target_variable, mask, max_target_len = batches

print("input_variable:", input_variable)
print("lengths:", lengths)
print("target_variable:", target_variable)
print("mask:", mask)
print("max_target_len:", max_target_len)


input_variable: tensor([[ 910, 5685,  393, 5080, 5685],
        [1643, 1352, 5685, 3318, 2724],
        [1643, 5024, 5824, 6343, 1662],
        [1643, 1563, 5589,  662, 6260],
        [1905, 3720, 4170, 1402, 2618],
        [ 662, 3961,  662, 5685, 6311],
        [1632, 1643, 3565, 5350, 1761],
        [ 662, 1643, 4170, 3720, 1643],
        [1974, 1643, 3355,  662,   49],
        [ 662,   37, 4162, 3814, 5024],
        [ 910, 1643, 2542,  662, 2618],
        [1643, 1643, 5126, 3814, 6705],
        [1643, 1643, 4455, 1883,   -1],
        [1643, 1643, 4770,  662,    0],
        [2715,   37,  347, 6469,    0],
        [5685, 1643, 5797, 3720,    0],
        [5824, 4326, 3016, 1590,    0],
        [3802, 1636, 4024, 1643,    0],
        [6574, 1643, 6405,   -1,    0],
        [ 662, 1643, 5627,    0,    0],
        [4910, 1643, 2211,    0,    0],
        [ 910, 5080, 2542,    0,    0],
        [1643,  662, 5126,    0,    0],
        [1643, 3090, 4455,    0,    0],
        [1643, 2724, 477

In [0]:
# #from pytorch
# import torch
# from torch.jit import script, trace
# import torch.nn as nn
# from torch import optim
# import torch.nn.functional as F

# class EncoderRNN(nn.Module):
#     def __init__(self, hidden_size, embedding, n_layers=1, dropout=0):
#         super(EncoderRNN, self).__init__()
#         self.n_layers = n_layers
#         self.hidden_size = hidden_size
#         self.embedding = embedding

#         # Initialize GRU; the input_size and hidden_size params are both set to 'hidden_size'
#         #   because our input size is a word embedding with number of features == hidden_size
#         self.gru = nn.GRU(hidden_size, hidden_size, n_layers,
#                           dropout=(0 if n_layers == 1 else dropout), bidirectional=True)

#     def forward(self, input_seq, input_lengths, hidden=None):
#         # Convert word indexes to embeddings
#         embedded = self.embedding(input_seq)
#         # Pack padded batch of sequences for RNN module
#         packed = nn.utils.rnn.pack_padded_sequence(embedded, input_lengths)
#         # Forward pass through GRU
#         outputs, hidden = self.gru(packed, hidden)
#         # Unpack padding
#         outputs, _ = nn.utils.rnn.pad_packed_sequence(outputs)
#         # Sum bidirectional GRU outputs
#         outputs = outputs[:, :, :self.hidden_size] + outputs[:, : ,self.hidden_size:]
#         # Return output and final hidden state
#         return outputs, hidden
    
# class Attn(nn.Module):
#     def __init__(self, method, hidden_size):
#         super(Attn, self).__init__()
#         self.method = method
#         if self.method not in ['dot', 'general', 'concat']:
#             raise ValueError(self.method, "is not an appropriate attention method.")
#         self.hidden_size = hidden_size
#         if self.method == 'general':
#             self.attn = nn.Linear(self.hidden_size, hidden_size)
#         elif self.method == 'concat':
#             self.attn = nn.Linear(self.hidden_size * 2, hidden_size)
#             self.v = nn.Parameter(torch.FloatTensor(hidden_size))

#     def dot_score(self, hidden, encoder_output):
#         return torch.sum(hidden * encoder_output, dim=2)

#     def general_score(self, hidden, encoder_output):
#         energy = self.attn(encoder_output)
#         return torch.sum(hidden * energy, dim=2)

#     def concat_score(self, hidden, encoder_output):
#         energy = self.attn(torch.cat((hidden.expand(encoder_output.size(0), -1, -1), encoder_output), 2)).tanh()
#         return torch.sum(self.v * energy, dim=2)

#     def forward(self, hidden, encoder_outputs):
#         # Calculate the attention weights (energies) based on the given method
#         if self.method == 'general':
#             attn_energies = self.general_score(hidden, encoder_outputs)
#         elif self.method == 'concat':
#             attn_energies = self.concat_score(hidden, encoder_outputs)
#         elif self.method == 'dot':
#             attn_energies = self.dot_score(hidden, encoder_outputs)

#         # Transpose max_length and batch_size dimensions
#         attn_energies = attn_energies.t()

#         # Return the softmax normalized probability scores (with added dimension)
#         return F.softmax(attn_energies, dim=1).unsqueeze(1)
# class LuongAttnDecoderRNN(nn.Module):
#     def __init__(self, attn_model, embedding, hidden_size, output_size, n_layers=1, dropout=0.1):
#         super(LuongAttnDecoderRNN, self).__init__()

#         # Keep for reference
#         self.attn_model = attn_model
#         self.hidden_size = hidden_size
#         self.output_size = output_size
#         self.n_layers = n_layers
#         self.dropout = dropout

#         # Define layers
#         self.embedding = embedding
#         self.embedding_dropout = nn.Dropout(dropout)
#         self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout=(0 if n_layers == 1 else dropout))
#         self.concat = nn.Linear(hidden_size * 2, hidden_size)
#         self.out = nn.Linear(hidden_size, output_size)

#         self.attn = Attn(attn_model, hidden_size)

#     def forward(self, input_step, last_hidden, encoder_outputs):
#         # Note: we run this one step (word) at a time
#         # Get embedding of current input word
#         embedded = self.embedding(input_step)
#         embedded = self.embedding_dropout(embedded)
#         # Forward through unidirectional GRU
#         rnn_output, hidden = self.gru(embedded, last_hidden)
#         # Calculate attention weights from the current GRU output
#         attn_weights = self.attn(rnn_output, encoder_outputs)
#         # Multiply attention weights to encoder outputs to get new "weighted sum" context vector
#         context = attn_weights.bmm(encoder_outputs.transpose(0, 1))
#         # Concatenate weighted context vector and GRU output using Luong eq. 5
#         rnn_output = rnn_output.squeeze(0)
#         context = context.squeeze(1)
#         concat_input = torch.cat((rnn_output, context), 1)
#         concat_output = torch.tanh(self.concat(concat_input))
#         # Predict next word using Luong eq. 6
#         output = self.out(concat_output)
#         output = F.softmax(output, dim=1)
#         # Return output and final hidden state
#         return output, hidden
    
# def maskNLLLoss(inp, target, mask):
#     nTotal = mask.sum()
#     crossEntropy = -torch.log(torch.gather(inp, 1, target.view(-1, 1)).squeeze(1))
#     loss = crossEntropy.masked_select(mask).mean()
#     loss = loss.to(device)
#     return loss, nTotal.item()

# MAX_LENGTH = 15
# def train(input_variable, lengths, target_variable, mask, max_target_len, encoder, decoder, embedding,
#           encoder_optimizer, decoder_optimizer, batch_size, clip, max_length=MAX_LENGTH):
#     # Zero gradients
#     encoder_optimizer.zero_grad()
#     decoder_optimizer.zero_grad()
#     # Set device options
#     input_variable = input_variable
#     lengths = lengths
#     target_variable = target_variable
#     mask = mask
#     # Initialize variables
#     loss = 0
#     print_losses = []
#     n_totals = 0
#     # Forward pass through encoder
#     encoder_outputs, encoder_hidden = encoder(input_variable, lengths)
#     # Create initial decoder input (start with SOS tokens for each sentence)
#     decoder_input = torch.LongTensor([[SOS_token for _ in range(batch_size)]])
#     decoder_input = decoder_input.to(device)
#     # Set initial decoder hidden state to the encoder's final hidden state
#     decoder_hidden = encoder_hidden[:decoder.n_layers]
#     # Determine if we are using teacher forcing this iteration
#     use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
#     # Forward batch of sequences through decoder one time step at a time
#     if use_teacher_forcing:
#         for t in range(max_target_len):
#             decoder_output, decoder_hidden = decoder(
#                 decoder_input, decoder_hidden, encoder_outputs
#             )
#             # Teacher forcing: next input is current target
#             decoder_input = target_variable[t].view(1, -1)
#             # Calculate and accumulate loss
#             mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
#             loss += mask_loss
#             print_losses.append(mask_loss.item() * nTotal)
#             n_totals += nTotal
#     else:
#         for t in range(max_target_len):
#             decoder_output, decoder_hidden = decoder(
#                 decoder_input, decoder_hidden, encoder_outputs
#             )
#             # No teacher forcing: next input is decoder's own current output
#             _, topi = decoder_output.topk(1)
#             decoder_input = torch.LongTensor([[topi[i][0] for i in range(batch_size)]])
#             decoder_input = decoder_input
#             # Calculate and accumulate loss
#             mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
#             loss += mask_loss
#             print_losses.append(mask_loss.item() * nTotal)
#             n_totals += nTotal

#     # Perform backpropatation
#     loss.backward()
#     # Clip gradients: gradients are modified in place
#     _ = nn.utils.clip_grad_norm_(encoder.parameters(), clip)
#     _ = nn.utils.clip_grad_norm_(decoder.parameters(), clip)
#     # Adjust model weights
#     encoder_optimizer.step()
#     decoder_optimizer.step()

#     return sum(print_losses) / n_totals

# def trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer, embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size, print_every, save_every, clip, corpus_name, loadFilename):

#     # Load batches for each iteration
#     training_batches = [batch2TrainData(voc, [random.choice(pairs) for _ in range(batch_size)])
#                       for _ in range(n_iteration)]

#     # Initializations
#     print('Initializing ...')
#     start_iteration = 1
#     print_loss = 0
#     if loadFilename:
#         start_iteration = checkpoint['iteration'] + 1

#     # Training loop
#     print("Training...")
#     for iteration in range(start_iteration, n_iteration + 1):
#         training_batch = training_batches[iteration - 1]
#         # Extract fields from batch
#         input_variable, lengths, target_variable, mask, max_target_len = training_batch
        
#         # Run a training iteration with batch
#         loss = train(input_variable, lengths, target_variable, mask, max_target_len, encoder,
#                      decoder, embedding, encoder_optimizer, decoder_optimizer, batch_size, clip)
#         print_loss += loss

#         # Print progress
#         if iteration % print_every == 0:
#             print_loss_avg = print_loss / print_every
#             print("Iteration: {}; Percent complete: {:.1f}%; Average loss: {:.4f}".format(iteration, iteration / n_iteration * 100, print_loss_avg))
#             print_loss = 0

#         # Save checkpoint
#         if (iteration % save_every == 0):
#             directory = os.path.join(save_dir, model_name, corpus_name, '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size))
#             if not os.path.exists(directory):
#                 os.makedirs(directory)
#             torch.save({
#                 'iteration': iteration,
#                 'en': encoder.state_dict(),
#                 'de': decoder.state_dict(),
#                 'en_opt': encoder_optimizer.state_dict(),
#                 'de_opt': decoder_optimizer.state_dict(),
#                 'loss': loss,
#                 'voc_dict': voc.__dict__,
#                 'embedding': embedding.state_dict()
#             }, os.path.join(directory, '{}_{}.tar'.format(iteration, 'checkpoint')))
        
# class GreedySearchDecoder(nn.Module):
#     def __init__(self, encoder, decoder):
#         super(GreedySearchDecoder, self).__init__()
#         self.encoder = encoder
#         self.decoder = decoder

#     def forward(self, input_seq, input_length, max_length):
#         # Forward input through encoder model
#         encoder_outputs, encoder_hidden = self.encoder(input_seq, input_length)
#         # Prepare encoder's final hidden layer to be first hidden input to the decoder
#         decoder_hidden = encoder_hidden[:decoder.n_layers]
#         # Initialize decoder input with SOS_token
#         decoder_input = torch.ones(1, 1, device=device, dtype=torch.long) * SOS_token
#         # Initialize tensors to append decoded words to
#         all_tokens = torch.zeros([0], device=device, dtype=torch.long)
#         all_scores = torch.zeros([0], device=device)
#         # Iteratively decode one word token at a time
#         for _ in range(max_length):
#             # Forward pass through decoder
#             decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden, encoder_outputs)
#             # Obtain most likely word token and its softmax score
#             decoder_scores, decoder_input = torch.max(decoder_output, dim=1)
#             # Record token and score
#             all_tokens = torch.cat((all_tokens, decoder_input), dim=0)
#             all_scores = torch.cat((all_scores, decoder_scores), dim=0)
#             # Prepare current token to be next decoder input (add a dimension)
#             decoder_input = torch.unsqueeze(decoder_input, 0)
#         # Return collections of word tokens and scores
#         return all_tokens, all_scores
    
# def evaluate(encoder, decoder, searcher, voc, sentence, max_length=MAX_LENGTH):
#     ### Format input sentence as a batch
#     # words -> indexes
#     indexes_batch = [indexesFromSentence(voc, sentence)]
#     # Create lengths tensor
#     lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
#     # Transpose dimensions of batch to match models' expectations
#     input_batch = torch.LongTensor(indexes_batch).transpose(0, 1)
#     # Use appropriate device
#     input_batch = input_batch.to(device)
#     lengths = lengths.to(device)
#     # Decode sentence with searcher
#     tokens, scores = searcher(input_batch, lengths, max_length)
#     # indexes -> words
#     decoded_words = [voc.index2word[token.item()] for token in tokens]
#     return decoded_words


# def evaluateInput(encoder, decoder, searcher, voc):
#     input_sentence = ''
#     while(1):
#         try:
#             # Get input sentence
#             input_sentence = input('> ')
#             # Check if it is quit case
#             if input_sentence == 'q' or input_sentence == 'quit': break
#             # Normalize sentence
#             input_sentence = normalizeString(input_sentence)
#             # Evaluate sentence
#             output_words = evaluate(encoder, decoder, searcher, voc, input_sentence)
#             # Format and print response sentence
#             output_words[:] = [x for x in output_words if not (x == 'EOS' or x == 'PAD')]
#             print('Bot:', ' '.join(output_words))

#         except KeyError:
#             print("Error: Encountered unknown word.")
            


# # Configure models
# model_name = 'cb_model'
# attn_model = 'dot'
# #attn_model = 'general'
# #attn_model = 'concat'
# hidden_size = 500
# encoder_n_layers = 2
# decoder_n_layers = 2
# dropout = 0.1
# batch_size = 64

# # Set checkpoint to load from; set to None if starting from scratch
# loadFilename = None
# checkpoint_iter = 4000
# #loadFilename = os.path.join(save_dir, model_name, corpus_name,
# #                            '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size),
# #                            '{}_checkpoint.tar'.format(checkpoint_iter))


# # Load model if a loadFilename is provided
# if loadFilename:
#     # If loading on same machine the model was trained on
#     checkpoint = torch.load(loadFilename)
#     # If loading a model trained on GPU to CPU
#     #checkpoint = torch.load(loadFilename, map_location=torch.device('cpu'))
#     encoder_sd = checkpoint['en']
#     decoder_sd = checkpoint['de']
#     encoder_optimizer_sd = checkpoint['en_opt']
#     decoder_optimizer_sd = checkpoint['de_opt']
#     embedding_sd = checkpoint['embedding']
#     voc.__dict__ = checkpoint['voc_dict']


# print('Building encoder and decoder ...')
# # Initialize word embeddings
# embedding = nn.Embedding(len(word2index), hidden_size)
# if loadFilename:
#     embedding.load_state_dict(embedding_sd)
# # Initialize encoder & decoder models
# encoder = EncoderRNN(hidden_size, embedding, encoder_n_layers, dropout)
# decoder = LuongAttnDecoderRNN(attn_model, embedding, hidden_size, len(word2index), decoder_n_layers, dropout)
# if loadFilename:
#     encoder.load_state_dict(encoder_sd)
#     decoder.load_state_dict(decoder_sd)
# # Use appropriate device
# encoder = encoder
# decoder = decoder
# print('Models built and ready to go!')


# import os
# # Configure training/optimization
# clip = 50.0
# teacher_forcing_ratio = 1.0
# learning_rate = 0.0001
# decoder_learning_ratio = 5.0
# n_iteration = 4000
# print_every = 1
# save_every = 500
# save_dir = os.path.join("data", "save")
# # Ensure dropout layers are in train mode
# encoder.train()
# decoder.train()

# # Initialize optimizers
# print('Building optimizers ...')
# encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
# decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate * decoder_learning_ratio)
# if loadFilename:
#     encoder_optimizer.load_state_dict(encoder_optimizer_sd)
#     decoder_optimizer.load_state_dict(decoder_optimizer_sd)

# # Run training iterations
# corpus_name = 'data'
# loadFilename = False
# print("Starting Training!")
# trainIters(model_name, word2index, chat_pair, encoder, decoder, encoder_optimizer, decoder_optimizer,
#            embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size,
#            print_every, save_every, clip, corpus_name, loadFilename)

# # Set dropout layers to eval mode
# encoder.eval()
# decoder.eval()

# # Initialize search module
# searcher = GreedySearchDecoder(encoder, decoder)